## Yellow Taxi Tripdata
### Bereinigung und Vorbereitung

#### Data Cleansing (Datenbereinigung)
Data Cleansing konzentriert sich darauf, schlechte Daten zu finden und zu beheben. Es geht um die Qualität der Daten. Stell dir vor, du korrigierst einen Text auf Rechtschreibfehler, fehlende Wörter oder doppelte Sätze.

Typische Aufgaben sind:

- Fehlende Werte auffüllen oder entfernen.

- Doppelte Datensätze löschen.

- Inkonsistente Einträge korrigieren (z. B. "USA" und "Vereinigte Staaten" zu standardisieren).

- Ausreißer und unsinnige Werte entfernen.

#### Data Wrangling (Datenaufbereitung)
Data Wrangling ist der übergeordnete, umfassende Prozess, um Rohdaten in eine nutzbare Form für die Analyse zu bringen. Es ist wie das gesamte Schreiben eines Berichts, nicht nur das Korrigieren. Es beinhaltet das Cleansing, geht aber noch weit darüber hinaus.

Typische Aufgaben sind:

- Das Cleansing der Daten.

- Das Verknüpfen mehrerer Datensätze (wie wir mit dem lookup_df machen werden).

- Das Umstrukturieren der Daten (z. B. Spalten in Zeilen umwandeln).

- Das Anreichern der Daten (z. B. neue Spalten für "Tag der Woche" erstellen).

**Zusammenfassend**: Das Ziel von Cleansing ist es, die Daten sauber und korrekt zu machen. Das Ziel von Wrangling ist es, die sauberen Daten bereit für die Analyse zu machen.

Zuerst widmen wir uns der Bereinigung des großen DataFrames. Wir korrigieren die Datentypen der Spalten, insbesondere der Datums- und Zeitangaben, kümmern uns um fehlende Werte und filtern offensichtliche Fehler wie Fahrten mit 0 Dollar Preis oder 0 Minuten Dauer heraus. Ziel ist ein sauberer, zuverlässiger Datensatz.

In [1]:
import pandas as pd
#import pyarrow
from fastparquet import ParquetFile

In [2]:
# Einlesen des Parquet-Files
pf = ParquetFile('NYTaxi-TripData/yellow_tripdata.parquet')
df = pf.to_pandas()

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2019-11-01 00:30:41,2019-11-01 00:32:25,1.0,0.0,1.0,N,145,145,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
1,1.0,2019-11-01 00:34:01,2019-11-01 00:34:09,1.0,0.0,1.0,N,145,145,2.0,2.5,0.5,0.5,0.00,0.0,0.3,3.80,0.0
2,2.0,2019-11-01 00:41:59,2019-11-01 00:42:23,1.0,0.0,1.0,N,193,193,1.0,2.5,0.5,0.5,0.95,0.0,0.3,4.75,0.0
3,2.0,2019-11-01 00:02:39,2019-11-01 00:02:51,1.0,0.0,1.0,N,193,193,1.0,2.5,0.5,0.5,0.95,0.0,0.3,4.75,0.0
4,2.0,2019-11-01 00:18:30,2019-11-01 00:18:39,2.0,0.0,1.0,N,226,226,2.0,2.5,0.0,0.5,0.00,0.0,0.3,3.30,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101246797 entries, 0 to 101246796
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           float64
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
dtypes: float64(13), int64(2), object(3)
memory usage: 13.6+ GB


### Legende
#### Bevor wir so richtig loslegen hier noch eine Übersicht über die Bedeutung der Spalten:
##### Die Einträge in einigen Spalten referenzieren Werte die auch nachfolgend angegeben sind.
- `VendorID`: Eine ID des Taxi-Anbieters (z. B. 1 für Creative Mobile Technologies, 2 für Curb Mobility).
    - 1: Creative Mobile Technologies, LLC
    - 2: VeriFone Inc.

- `tpep_pickup_datetime`: Das Datum und die Uhrzeit, wann die Taxifahrt begonnen hat.

- `tpep_dropoff_datetime`: Das Datum und die Uhrzeit, wann die Fahrt beendet wurde.

- `passenger_count`: Die Anzahl der Passagiere auf der Fahrt.

- `trip_distance`: Die zurückgelegte Entfernung in Meilen.

- `PULocationID`: Die ID der Zone, in der die Fahrt begonnen hat. (Zone-Lookup-Frame)

- `DOLocationID`: Die ID der Zone, in der die Fahrt geendet hat.  (Zone-Lookup-Frame)

- `RatecodeID`: Ein Code für den Tarif der Fahrt (z. B. 1 = Standard, 2 = Fahrt zum JFK-Flughafen).
    - 1: Standard rate
    - 2: JFK
    - 3: Newark
    - 4: Nassau or Westchester
    - 5: Negotiated fare
    - 6: Group ride
- `payment_type`: Der Zahlungs-Typ (z. B. 1 = Kreditkarte, 2 = Barzahlung).
    - 1: Credit card
    - 2: Cash
    - 3: No charge
    - 4: Dispute
    - 5: Unknown
    - 6: Voided trip

- `fare_amount`: Der reine Fahrpreis (ohne Extras).

- `congestion_surcharge`: Eine eventuell erhobene Staugebühr.

#### store_and_fwd_flag
Hier noch eine Erklärung zu der Spalte store_and_fwd_flag:
Das `store_and_fwd_flag` sagt aus, ob die Daten einer Taxifahrt sofort an den Server des Anbieters gesendet werden konnten oder nicht.

Wenn das Signal im Taxi schlecht war, wurden die Daten lokal auf dem Taxameter gespeichert (store) und erst später, als die Verbindung wieder da war, weitergeleitet (forward).
- 'Y': Die Daten wurden erst später weitergeleitet.
- 'N': Die Daten wurden sofort gesendet.

Für eine erste, einfache Analyse ist der store_and_fwd_flag vielleicht nicht das wichtigste Feature.

Aber es kann wichtig sein, um die Datenqualität zu beurteilen. Wenn viele Fahrten dieses Flag haben, könnte das auf ein schlechtes GPS-Signal hindeuten, was wiederum die Genauigkeit der Fahrzeiten und der Entfernungen beeinflussen kann. Das wäre dann ein wichtiger Punkt, den man bei der Interpretation der Ergebnisse berücksichtigen müsste.

Auch könnte es sein, dass es für Machine-Learning-Modelle wichtig ist. Vielleicht gibt es einen Zusammenhang zwischen dem Flag und der Höhe des Trinkgeldes oder der Fahrtdauer. Aber das sind eher fortgeschrittene Themen.

Kurz gesagt: Es ist ein kleiner, aber feiner Hinweis auf die Umstände, unter denen die Daten gesammelt wurden.

#### Datentypen korrigieren
Der erste Schritt ist, die Datentypen der Datums- und Zeitspalten korrekt einzustellen. Wie wir besprochen haben, müssen die Spalten tpep_pickup_datetime und tpep_dropoff_datetime in das datetime-Format umgewandelt werden.

In [5]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101246797 entries, 0 to 101246796
Data columns (total 18 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               float64       
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           float64       
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
dtypes: datetime64[ns](

In [6]:
# df.isnull().sum <- ergibt eine unübersichliche Ausgabe

# wir sind nur an Spalten interessiert, die mindestens 1 fehlenden Wert haben.
df.isnull().sum()[df.isnull().sum() > 0]

VendorID                 527484
passenger_count          527484
RatecodeID               527484
store_and_fwd_flag       527484
payment_type             527484
congestion_surcharge    4855981
dtype: int64

Für die `congestion_surcharge`Wert gehen wir davon aus, dass für die Fahrt keine Staugebühr erhoben wurde.
Wir ersetzen die fehlenden Werte (NaN) durch eine 0

In [7]:
# Ersetze alle fehlenden Werte in der congestion_surcharge-Spalte durch 0
#df['congestion_surcharge'].fillna(0, inplace=True) <-- erzeugt eine Warnung

# So geht es ohne Warnmeldung
df['congestion_surcharge'] = df['congestion_surcharge'].fillna(0)

# Nochmal die Ausgabe zur Kontrolle
df.isnull().sum()[df.isnull().sum() > 0]


VendorID              527484
passenger_count       527484
RatecodeID            527484
store_and_fwd_flag    527484
payment_type          527484
dtype: int64

#### Interpretation der Fehlwerte
Die Tatsache, dass alle fünf Spalten exakt die gleiche Anzahl an fehlenden Werten haben, ist ein sehr starkes Signal. Es ist höchst unwahrscheinlich, dass das Zufall ist.

##### Interpretation:
Diese 527.484 fehlenden Werte betreffen wahrscheinlich die gleichen Fahrten. Es scheint, als wären für diese Trips aus irgendeinem Grund die Daten für alle diese Spalten auf einmal nicht erfasst worden.

##### Nächster Schritt:
Da uns für diese Fahrten so viele wichtige Informationen fehlen, sind die Zeilen für unsere Analyse wahrscheinlich nicht nutzbar. Die beste Methode, um damit umzugehen, ist, diese Zeilen einfach aus dem Datensatz zu entfernen.

In [8]:
# (vorher)
df.shape

(101246797, 18)

In [9]:
df = df.dropna(subset=['VendorID', 'passenger_count', 'RatecodeID', 'store_and_fwd_flag', 'payment_type'])

In [10]:
# (nachher) - Die Differenz sollte wieder 527.484 ergeben...
df.shape

(100719313, 18)

- Die Methode `dropna()` entfernt Zeilen mit fehlenden Werten (NaN, None, etc.)
- Der Parameter `subset` gibt an, dass nur die genannten Spalten auf fehlende Werte überprüft werden sollen
- Es werden nur Zeilen entfernt, die in mindestens einer der folgenden Spalten fehlende Werte haben:
    - 'VendorID'
    - 'passenger_count'
    - 'RatecodeID'
    - 'store_and_fwd_flag'
    - 'payment_type'

- Das Ergebnis wird dem ursprünglichen DataFrame `df` zugewiesen, was bedeutet, dass der originale DataFrame überschrieben wird

Diese Operation ist typisch für die Datenbereinigung, wobei unvollständige Datensätze entfernt werden, bevor weitere Analysen durchgeführt werden. Alterntiv können wir natürlich auch fehlende Daten einfach ersetzten - dazu werden wir später noch mehr erfahren.




#### Filtern unnötiger Datensätze
Wir filtern den Datensatz so, dass nur noch Fahrten übrig bleiben, bei denen sowohl die Entfernung als auch der Fahrpreis größer als 0 sind.

In [11]:
# Filtert den DataFrame, um nur Fahrten mit trip_distance und fare_amount größer 0 zu behalten
df = df[(df['trip_distance'] > 0) & (df['fare_amount'] > 0)]
df.shape

(99574711, 18)

### Speichern & Feierabend
Natürlich wollen wir unseren schönen, sauberen und bereinigten Datensatz für zukünftige Missionen speichern!

In [12]:
# Speichern des aktuellen Dataframes
df.to_parquet('NYTaxi-TripData/new_york_taxi_cleaned.parquet', index=False)